In [ ]:
from PIL import Image

def make_background_transparent(img_path, output_path, threshold=50):
    """
    Convert the black background of an image to transparent.
    
    Parameters:
    - img_path: Path to the input image.
    - output_path: Path to save the output image.
    - threshold: The RGB value below which colors will be made transparent.
    """
    #img_path = "threestudio/load/images/shark_right.png"
    img = Image.open(img_path).convert("RGBA")
    #output_path = "threestudio/load/images/shark_right."
    datas = img.getdata()

    new_data = []
    for item in datas:
        # Change all black (also shades of blacks)
        # pixels to transparent
        if item[0] < threshold and item[1] < threshold and item[2] < threshold:
            new_data.append((255, 255, 255, 0))
        else:
            new_data.append(item)

    img.putdata(new_data)
    img.save(output_path, "PNG")

# Example usage
make_background_transparent("threestudio/load/images/shark_right.png", "threestudio/load/images/shark_right_trans.png")


In [ ]:
from controlnet_aux import CannyDetector, NormalBaeDetector
import cv2
import numpy as np

preprocessor = CannyDetector()

def prepare_image_cond(cond_rgb: Float[Tensor, "B H W C"]):
    cond_rgb = (
        (cond_rgb[0].detach().cpu().numpy() * 255).astype(np.uint8).copy()
    )
    blurred_img = cv2.blur(cond_rgb, ksize=(5, 5))
    detected_map = self.preprocessor(
        blurred_img, self.cfg.canny_lower_bound, self.cfg.canny_upper_bound
    )
    control = (
        torch.from_numpy(np.array(detected_map)).float() / 255.0
    )
    print(control.shape)
    control = control.unsqueeze(-1).repeat(1, 1, 3)
    control = control.unsqueeze(0)
    control = control.permute(0, 3, 1, 2)
    return control

prepare_image_cond("/nvme/yyh/threestudio/load/images/shark_instruct_64.png")